In [1377]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
# from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from scipy import stats
import lightgbm as lgb

In [1378]:
df = pd.read_csv("sources/CreditPrediction.csv")

In [1379]:

z_dataset = pd.DataFrame()
df = df.drop(['CLIENTNUM', 'Unnamed: 19', 'Total_Ct_Chng_Q4_Q1', 'Months_on_book', 'Education_Level', 'Marital_Status', ], axis=1)


z_dataset['Total_Trans_Amt'] = np.abs(stats.zscore(df['Total_Trans_Amt']))
outlAmt = list(np.where(np.abs(z_dataset['Total_Trans_Amt']) > 3)[0])
df = df.drop(df.index[outlAmt])

z_dataset['Total_Trans_Ct'] = np.abs(stats.zscore(df['Total_Trans_Ct']))
outlct = list(np.where(np.abs(z_dataset['Total_Trans_Ct']) > 3)[0])
df = df.drop(df.index[outlct])

missing_values = ["NaN", "Unknown", ""]
df.replace(missing_values, np.nan, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9763 entries, 0 to 10166
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              9763 non-null   float64
 1   Gender                    9568 non-null   object 
 2   Dependent_count           9763 non-null   int64  
 3   Income_Category           8673 non-null   object 
 4   Card_Category             7913 non-null   object 
 5   Total_Relationship_Count  9744 non-null   float64
 6   Months_Inactive_12_mon    9763 non-null   int64  
 7   Contacts_Count_12_mon     9763 non-null   int64  
 8   Credit_Limit              9763 non-null   float64
 9   Total_Revolving_Bal       9763 non-null   int64  
 10  Total_Amt_Chng_Q4_Q1      9763 non-null   float64
 11  Total_Trans_Amt           9763 non-null   int64  
 12  Total_Trans_Ct            9763 non-null   int64  
 13  Avg_Utilization_Ratio     9763 non-null   float64
dtypes: float64(5

In [1380]:
df['Income_Category'] = df['Income_Category'].map({"Less than $40K":0, "$40K - $60K":1, "$60K - $80K":2 ,"$80K - $120K":3, "$120K +":4})
df['Card_Category'] = df['Card_Category'].map({"Blue":0, "Silver":1, "Gold":2, "Platinum":3})
df['Gender'] = df['Gender'].map({"M":0, "F":1})

In [1381]:
# from sklearn.metrics import silhouette_score
# import matplotlib.pyplot as plt

# # مقداردهی اولیه داده‌های مفقود با میانگین
# imputer = SimpleImputer(strategy='mean')
# data_imputed = imputer.fit_transform(df)

# # روش Elbow
# sse = []
# k_range = range(1, 11)
# for k in k_range:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(data_imputed)
#     sse.append(kmeans.inertia_)

# plt.figure(figsize=(10, 6))
# plt.plot(k_range, sse, marker='o')
# plt.xlabel('تعداد کلاسترها')
# plt.ylabel('SSE')
# plt.title('روش Elbow برای تعیین تعداد بهینه کلاسترها')
# plt.show()

In [1382]:
# # روش Silhouette
# silhouette_scores = []
# for k in range(2, 11):
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     clusters = kmeans.fit_predict(data_imputed)
#     silhouette_scores.append(silhouette_score(data_imputed, clusters))

# plt.figure(figsize=(10, 6))
# plt.plot(range(2, 11), silhouette_scores, marker='o')
# plt.xlabel('تعداد کلاسترها')
# plt.ylabel('Silhouette Score')
# plt.title('روش Silhouette برای تعیین تعداد بهینه کلاسترها')
# plt.show()

In [1383]:
# from sklearn.experimental import enable_iterative_imputer  # Needed to enable IterativeImputer
# from sklearn.impute import IterativeImputer

# mice_imputer = IterativeImputer(max_iter=20, random_state=42)

# # Fit and transform the data
# df_imputed = mice_imputer.fit_transform(df)

# # Convert the imputed data back to a DataFrame
# df_imputed = pd.DataFrame(df_imputed, columns=df.columns)
# df = df_imputed
# df

In [1384]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# مرحله 2: مقداردهی اولیه داده‌های مفقود با میانگین
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(df)

# تبدیل داده‌های پر شده به DataFrame
df_imputed = pd.DataFrame(data_imputed, columns=df.columns)

# نمایش داده‌های پر شده با میانگین


# مرحله 3: اجرای الگوریتم K-means
kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(data_imputed)

# اضافه کردن برچسب خوشه به داده‌ها
df_imputed['cluster'] = clusters



# مرحله 4: جایگزینی داده‌های مفقود
for feature in df.columns:
    if df[feature].isnull().any():
        for cluster in range(kmeans.n_clusters):
            # محاسبه میانگین ویژگی برای هر خوشه
            cluster_mean = df_imputed[df_imputed['cluster'] == cluster][feature].mean()
            # جایگزینی داده‌های مفقود با میانگین خوشه
            df_imputed.loc[(df_imputed['cluster'] == cluster) & (df[feature].isnull()), feature] = cluster_mean

# حذف ستون برچسب خوشه
df_final = df_imputed.drop(columns=['cluster'])

# نمایش داده‌های نها

df = df_final
df_imputed

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio,cluster
0,45.0,0.0,3.0,2.000000,0.000000,5.0,1.0,3.0,12691.0,777.0,1.335,1144.0,42.0,0.061,4
1,49.0,1.0,5.0,0.000000,0.000000,6.0,1.0,2.0,8256.0,864.0,1.541,1291.0,33.0,0.105,1
2,51.0,0.0,3.0,3.000000,0.000000,4.0,1.0,0.0,3418.0,0.0,2.594,1887.0,20.0,0.000,5
3,40.0,1.0,4.0,0.000000,0.000000,3.0,4.0,1.0,3313.0,2517.0,1.405,1171.0,20.0,0.760,5
4,40.0,0.0,3.0,2.000000,0.014805,5.0,1.0,0.0,4716.0,0.0,2.175,816.0,28.0,0.000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,35.0,0.0,3.0,3.000000,0.000000,5.0,3.0,4.0,13590.0,1528.0,0.728,2137.0,52.0,0.112,4
9759,46.0,1.0,1.0,0.000000,0.000000,2.0,2.0,1.0,2029.0,1074.0,0.514,4802.0,90.0,0.529,6
9760,52.0,0.0,0.0,2.000000,0.000000,3.0,2.0,4.0,2742.0,2184.0,0.592,3829.0,72.0,0.796,6
9761,39.0,1.0,1.0,1.334025,0.076962,1.0,3.0,2.0,2751.0,1158.0,0.821,4861.0,82.0,0.421,6


In [1385]:
from scipy import stats

# Box-Cox transformation
box_cox_transformed, _ = stats.boxcox(df['Customer_Age'])
df['Customer_Age'] = box_cox_transformed

box_cox_transformed, _ = stats.boxcox(df['Total_Trans_Amt'])
df['Total_Trans_Amt'] = box_cox_transformed

df

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
0,1.864972,0.0,3.0,2.000000,0.000000,5.0,1.0,3.0,12691.0,777.0,1.335,6.586399,42.0,0.061
1,1.881052,1.0,5.0,0.000000,0.000000,6.0,1.0,2.0,8256.0,864.0,1.541,6.691856,33.0,0.105
2,1.888403,0.0,3.0,3.000000,0.000000,4.0,1.0,0.0,3418.0,0.0,2.594,7.021395,20.0,0.000
3,1.841732,1.0,4.0,0.000000,0.000000,3.0,4.0,1.0,3313.0,2517.0,1.405,6.606768,20.0,0.760
4,1.841732,0.0,3.0,2.000000,0.014805,5.0,1.0,0.0,4716.0,0.0,2.175,6.290346,28.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,1.813910,0.0,3.0,3.000000,0.000000,5.0,3.0,4.0,13590.0,1528.0,0.728,7.128888,52.0,0.112
9759,1.869179,1.0,1.0,0.000000,0.000000,2.0,2.0,1.0,2029.0,1074.0,0.514,7.822153,90.0,0.529
9760,1.891926,0.0,0.0,2.000000,0.000000,3.0,2.0,4.0,2742.0,2184.0,0.592,7.629354,72.0,0.796
9761,1.836580,1.0,1.0,1.334025,0.076962,1.0,3.0,2.0,2751.0,1158.0,0.821,7.832527,82.0,0.421


In [1386]:
features = df.columns.difference(['Credit_Limit'])  # All columns except 'SalePrice'
target = 'Credit_Limit'

# Convert categorical features to strings

categorical_features = df[features].select_dtypes(include=['object']).columns

for feature in categorical_features:

    df[feature] = df[feature].astype(str)
 
# Split data into features and target

X = df[features]
# StandardScaler
scaler = MinMaxScaler()

# Fit the scaler to the features and transform them
X_scaled = scaler.fit_transform(X)



y = df[target]
# Split the data into training and testing sets

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Create the RFE object and rank each feature
model = LogisticRegression()
rfe = RFE(model, n_features_to_select=2)
fit = rfe.fit(X, y)
selected_features = X.columns[fit.support_]
print("Selected Features (RFE):", selected_features)
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# # Find categorical features for CatBoost

# categorical_features_indices = np.where(X.dtypes == 'object')[0]


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [1387]:
# Check for missing values

missing_values = df.isnull().sum().sort_values(ascending=False)

missing_values = missing_values[missing_values > 0]

print("\nColumns with missing values:\n", missing_values)


Columns with missing values:
 Series([], dtype: int64)


In [1388]:
lgb_reg = lgb.LGBMRegressor()
lgb_reg.fit(X_train, y_train)

y_pred = lgb_reg.predict(X_test)
print(mean_squared_error(y_test, y_pred))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000110 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1257
[LightGBM] [Info] Number of data points in the train set: 7810, number of used features: 13
[LightGBM] [Info] Start training from score 8380.299964
5110112.89735328


In [1389]:
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)

test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)
 
# Train the CatBoost model

model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.1,
    eval_metric='RMSE',
    logging_level='Verbose',  # Can be 'Silent' to reduce output
    use_best_model=True, nan_mode='Max'
    
)

# Train the model with evaluation
model.fit(
    train_pool,
    eval_set=test_pool,
    early_stopping_rounds=50  # Stops if validation RMSE doesn't improve for 50 rounds
)

0:	learn: 8322.9376635	test: 8410.0322184	best: 8410.0322184 (0)	total: 2.93ms	remaining: 29.3s
1:	learn: 7751.2433501	test: 7809.0975834	best: 7809.0975834 (1)	total: 5.77ms	remaining: 28.8s
2:	learn: 7202.8055155	test: 7217.2488600	best: 7217.2488600 (2)	total: 8.49ms	remaining: 28.3s
3:	learn: 6736.3892395	test: 6705.2470369	best: 6705.2470369 (3)	total: 10.8ms	remaining: 27s
4:	learn: 6306.2403818	test: 6236.3191355	best: 6236.3191355 (4)	total: 13ms	remaining: 26.1s
5:	learn: 5963.5931770	test: 5866.9920841	best: 5866.9920841 (5)	total: 15.6ms	remaining: 26.1s
6:	learn: 5616.7082639	test: 5490.3032040	best: 5490.3032040 (6)	total: 18.2ms	remaining: 25.9s
7:	learn: 5312.6046844	test: 5155.4569835	best: 5155.4569835 (7)	total: 21ms	remaining: 26.2s
8:	learn: 5053.2111809	test: 4879.9016569	best: 4879.9016569 (8)	total: 23.3ms	remaining: 25.8s
9:	learn: 4838.5361533	test: 4639.5847045	best: 4639.5847045 (9)	total: 25.9ms	remaining: 25.9s
10:	learn: 4623.1154772	test: 4399.5995408	bes

In [1308]:
# Make predictions on the test set
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(test_pool)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mse

5257403.788225978